# Linear regression

## En esta parte, voy a ir describiendo el proceso y los pasos necesarios para construir un riguroso modelo de Regresion Lineal que nos permita predecir los valores de la futura temperatura media diaria basandonos en el dataSet que ya tenemos preparado
## Usare dos de las librerias mas importantes de python para la industria del Machine Learning: Scikit-Learn y StatsModels 

### Abrimos el datafram exportado de la primera parte

In [1]:
import pickle
with open('end-part1_df.pkl', 'rb') as fp:
    df = pickle.load(fp)

### Utilizaremos un modelo de regresion lineal con el algoritmo de MCO (minimos cuadrados ordinarios)

[Informacion sobre que es la regresion lineal](https://es.wikipedia.org/wiki/Regresi%C3%B3n_lineal)

[Informacion sobre el método utilizado para encontrar los parámetros poblacionales en un modelo de regresión lineal](https://es.wikipedia.org/wiki/M%C3%ADnimos_cuadrados_ordinarios)

QUIZA dar mas explicaciones